## Convolution Neuron Network

In this section, you wil be implement a simple Convolution Neuron Network(CNN) via tensorflow. You will be playing with different different architecture of the CNN and see how it affect of the performance of the CNN. In order to finish this section, you dont need to read ducomentation from tensorflow if you are not familiar with it, we will set up all the background for you and all you need to do is to see different how architectural choice affect the performance of the CNN

Let's start by importing the libaries and setting up the model parameters. In this Pset, you will be working with the MNIST data sets, which is a set of images of handwriting digits, 0-9, you will be using CNN to classify them

By now, you should have all the required libararies installed on your VM by the TA. If you see errors from the box below, please contact TA for support

In [1]:
# Import MINST data
import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
import tensorflow as tf
import pdb

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


Below, the "dropout" is a technique to regulate the weights during the training to prevent the model overfitting. 

Tradiationlly, when solving an optimization problem, we use the Lagrangian multipliers and add them to the cost function. However, in the CNN, in practice, there are millions weights, therefore, in order to regulate the weights, at each layer, people randomly select a few number of neurons and "drop" them from the training to prevent overfiting.

In [2]:
# Parameters
learning_rate = 0.001
training_iters = 100000
# decsion the Batch size for the SGD algorithm
batch_size = 128
#how often do we show the output
display_step = 20
# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

In [3]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [4]:
#define covolution, ReLU layer and max pooling layer
def conv2d(img, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, w, strides=[1, 1, 1, 1], 
                                                  padding='VALID'),b))

def max_pool(img, k):
    return tf.nn.max_pool(img, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='VALID')


In the box below, we defined a CNN with 1 layer or convolution and 1 layer of max pool architecture for you. Please pay attention to how the variable is passed from "Apply Dropout" to the "Fully connected layer". Here you just need to run this code and later, we will ask to added one more convulition/ReLU layer and max pooling layer

In [5]:
def conv_net(_X, _weights, _biases, _dropout):
    # Reshape input picture
    _X = tf.reshape(_X, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(_X, _weights['wc1'], _biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = max_pool(conv1, k=3)
    # Apply Dropout
    conv1 = tf.nn.dropout(conv1, _dropout)

    # Convolution Layer
    #conv2 = conv2d(conv1, _weights['wc2'], _biases['bc2'])
    # Max Pooling (down-sampling)
    #conv2 = max_pool(conv2, k=2)
    # Apply Dropout
    #conv2 = tf.nn.dropout(conv2, _dropout)

    # Fully connected layer
    # Reshape conv2 output to fit dense layer input
    dense1 = tf.reshape(conv1, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    # Relu activation
    dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, _weights['wd1']), _biases['bd1']))
    # Apply Dropout
    dense1 = tf.nn.dropout(dense1, _dropout) # Apply Dropout

    # Output, class prediction
    out = tf.add(tf.matmul(dense1, _weights['out']), _biases['out'])
    return out



# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])), 
    # fully connected, 8*8*32 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([8*8*32, 1024])), 
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes])) 
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}



In [6]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# Initializing the variables
init = tf.initialize_all_variables()

In [7]:
# Train the model and get the accuracy score
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        if step == 1:
            print "Training started:"
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
        
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"
    # Calculate accuracy for 256 mnist test images
    print "Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], 
                                                             y: mnist.test.labels[:256], 
                                                             keep_prob: 1.})

Training started:
Iter 2560, Minibatch Loss= 769.969849, Training Accuracy= 0.38281
Iter 5120, Minibatch Loss= 332.464722, Training Accuracy= 0.64844
Iter 7680, Minibatch Loss= 469.839355, Training Accuracy= 0.70312
Iter 10240, Minibatch Loss= 198.065796, Training Accuracy= 0.80469
Iter 12800, Minibatch Loss= 233.267090, Training Accuracy= 0.75000
Iter 15360, Minibatch Loss= 135.702972, Training Accuracy= 0.85938
Iter 17920, Minibatch Loss= 89.567581, Training Accuracy= 0.86719
Iter 20480, Minibatch Loss= 103.814590, Training Accuracy= 0.89844
Iter 23040, Minibatch Loss= 87.189461, Training Accuracy= 0.91406
Iter 25600, Minibatch Loss= 88.774643, Training Accuracy= 0.84375
Iter 28160, Minibatch Loss= 112.390198, Training Accuracy= 0.85156
Iter 30720, Minibatch Loss= 69.709091, Training Accuracy= 0.91406
Iter 33280, Minibatch Loss= 63.219002, Training Accuracy= 0.91406
Iter 35840, Minibatch Loss= 23.709538, Training Accuracy= 0.95312
Iter 38400, Minibatch Loss= 24.275436, Training Accur

# Your coding starts here

Now add one more convolution/ReLU layer and max pooling layer in the box below. Then run the flowing code to train your model again and compare the accucracy with the CNN with only 1 convolution/ReLU layer

Hint: you need to re-calculate the stride size based on the equation from the course slide

In [ ]:
#define covolution, ReLU layer and max pooling layer
def conv2d(img, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, w, strides=[1, 1, 1, 1], 
                                                  padding='SAME'),b))

def max_pool(img, k):
    return tf.nn.max_pool(img, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [ ]:
def conv_net(_X, _weights, _biases, _dropout):
    # Reshape input picture
    _X = tf.reshape(_X, shape=[-1, 28, 28, 1])

    # Convolution Layer 1
    conv1 = conv2d(_X, _weights['wc1'], _biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = max_pool(conv1, k=)## You need to re-cacluate the stride here, which is value of k
    # Apply Dropout
    conv1 = tf.nn.dropout(conv1, _dropout)

    ## your code here:
    
    # Convolution Layer 2
    
    # Max Pooling (down-sampling)
    
    # Apply Dropout
    

    # Fully connected layer
    # Reshape conv1 output to fit dense layer input
    ## remember to change your input
    
    dense1 = tf.reshape(conv1, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    # Relu activation
    dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, _weights['wd1']), _biases['bd1']))
    # Apply Dropout
    dense1 = tf.nn.dropout(dense1, _dropout) # Apply Dropout

    # Output, class prediction
    out = tf.add(tf.matmul(dense1, _weights['out']), _biases['out'])
    return out

Since you add one more layer, then you will need to change the number of weights.

Hint: you will need to add weights and bias for the new layer

In [ ]:

# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),  
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])), 
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes])) 
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# You don't need to edit the following code, just run them

In [ ]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# Initializing the variables
init = tf.initialize_all_variables()

In [ ]:
# Train the model and get the accuracy score
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"
    # Calculate accuracy for 256 mnist test images
    print "Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], 
                                                             y: mnist.test.labels[:256], 
                                                             keep_prob: 1.})

Congratulations, you are all set.